# INITIAL SETUP

In [1]:
# Import dependencies
import random
import copy
import numpy as np
import time

In [2]:
# custom functions

# Defining dictionary layout of decks
def new_deck():
    return {"suits": [],
            "values": [],
            "ids": []}

# Takes an input, and randomly selects an index from the original, to one by one assemble a shuffled deck
def randomized_shuffling(deck):
    # Create a copy of the original deck to work with
    bin_deck = copy.deepcopy(deck)

    # Create a new deck to store the shuffled cards
    master_deck = new_deck()

    # Randomize the deck
    while len(bin_deck["suits"]) > 0:
        # Pick a random index
        random_index = random.randint(0, len(bin_deck["suits"]) - 1)

        # Add the card at the random index to the master deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].append(bin_deck[key].pop(random_index))

    return master_deck

# Takes an input, and shuffles is in an alternating pattern
def interlaced_shuffling(deck):
    # Initial Setup
    temp_deck = copy.deepcopy(deck)

    # Randomize the deck a random number of times
    shuffle_count = random.randint(10, 20)
    for _ in range(shuffle_count):
        # Create a temporary deck to store the shuffled cards
        master_deck = new_deck()

        # Interlace shuffle
        half = len(temp_deck["suits"]) // 2
        for card in range(half):
            for key in ["suits", "values", "ids"]:
                master_deck[key].append(temp_deck[key][half + card])
                master_deck[key].append(temp_deck[key][card])

    return master_deck

# Takes 2 inputs, and seperates them accross 52 bins in a random order before reassembling by bin
def rotary_shuffling(master_deck, discard_deck):
    # Port and merge to temp dictionary
    temp_deck = new_deck()
    for key in temp_deck:
        temp_deck[key].extend(discard_deck[key])
        temp_deck[key].extend(master_deck[key])

    # Resets
    master_deck = new_deck()
    discard_deck = new_deck()

    # Rotary shuffler reset / setup
    rotary_shuffler = {}
    slot_bins = list(range(52))
    open_bins = list(range(52))

    for id in range(52):
        # Building out dictionary with 52 bins
        slot_name = f"Bin {id}"
        rotary_shuffler[slot_name] = new_deck().copy()

    # Loop until all cards are binned
    while temp_deck["suits"]:
        # Randomized target
        current_slot = random.choice(open_bins)
        slot_name = f"Bin {current_slot}"

        # Checking if bin has room, and transferring values if able
        if len(rotary_shuffler[slot_name]["suits"]) <= 8:
            for key in ["suits", "values", "ids"]:
                rotary_shuffler[slot_name][key].append(temp_deck[key].pop(0))
            if len(rotary_shuffler[slot_name]["suits"]) == 8:
                open_bins.remove(current_slot)

    # Dump bins to new master_deck
    opposite_slot = random.choice(slot_bins) + 25

    while slot_bins:
        # Adjustments to stay within correct index range
        opposite_slot %= 52
        slot_name = f"Bin {opposite_slot}"

        # Transferring values back to master_deck
        for key in ["suits", "values", "ids"]:
            master_deck[key].extend(rotary_shuffler[slot_name][key])

        del rotary_shuffler[slot_name]
        slot_bins.remove(opposite_slot)

        # Move to next bin
        opposite_slot += 1

    # Return the shuffled master_deck and the emptied/reset discard_deck
    return master_deck, discard_deck

In [3]:
#Inital Deck build
# Unicode icons
spade = '\u2660'
club = '\u2663'
heart = '\u2665'
diamond = '\u2666'

#setting up unique features
suits = [spade,diamond,club,heart]
faces = ["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
values = [11,2,3,4,5,6,7,8,9,10,10,10,10]

# initial setup
raw_deck = {"suits": [],
            "values": []}

# loop to initalize all unique combinations in both bins
for suit_index,suit in enumerate(suits):
    # forward iteration for first 2 suits
    if suit_index < 2:
        for face_index, face in enumerate(faces):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[face_index]]
    # reverse iteration for last 2 suits
    else:
        for face_index, face in enumerate(reversed(faces)):
            raw_deck["suits"] += [suit + face]
            raw_deck["values"] += [values[-face_index-1]]
            
# Now building a deck of 8 decks in new deck order
raw_master = new_deck()

# loop to make master deck using 8 raw decks
for deck in range(8):
    raw_master["suits"] += raw_deck["suits"]
    raw_master["values"] += raw_deck["values"]
    
# loop to create unique ids for each of the cards made in the master deck
for id in range(len(raw_master["suits"])):
    raw_master["ids"] += [id]

In [4]:
# doing a random shuffle first
master_deck = randomized_shuffling(raw_master)
# then doing an interlaced shuffle to further randomize
master_deck = interlaced_shuffling(master_deck)

# initializing discard deck
discard_deck = new_deck()

# SIMULATION

In [5]:
print("This simulation allows you to auto-play multiple rounds based on a specified hand limit, and rounds to be played.")
print("Through the multiple rounds, the a deck with a static order is simulated during drawn and shuffled cards.")

This simulation allows you to auto-play multiple rounds based on a specified hand limit, and rounds to be played.
Through the multiple rounds, the a deck with a static order is simulated during drawn and shuffled cards.


In [6]:
hand_attempts = 0
round_attempts = 0

#_____________________________________________________
# Loop to obtain hand limit
while hand_attempts < 10:
    hand_hold = input('What would you like the player limit to be (dealer holds at 17)?  ')
    
    try:
        number = int(hand_hold)
        if number > 0:
            print(f"Thank you! Player will draw to {hand_hold}.")
            hand_hold = int(hand_hold)
            break  # loop exit
        else:
            print("Please enter a positive number.")
            hand_attempts  += 1
    except ValueError:
        print("That's not a valid number. Try again.")
        hand_attempts += 1
        
#_____________________________________________________
# Error exit      
if hand_attempts >= 10:
    print("Looks like there's some difficulty making a selection. Let's just set the limit to a standard 17.")
    hand_hold = 17
    
#_____________________________________________________
# Loop to obtain hand limit
while round_attempts < 10:
    print("")
    round_hold = input('How many rounds would you like simulated?  ')
    
    try:
        number = int(hand_hold)
        if number > 0:
            print(f"Thank you! {round_hold} rounds will be simulated")
            round_hold = int(round_hold)
            break  # loop exit
        else:
            print("Please enter a positive number.")
            round_attempts  += 1
    except ValueError:
        print("That's not a valid number. Try again.")
        round_attempts += 1
        
#_____________________________________________________
# Error exit          
if round_attempts >= 10:
    print("Looks like there's some difficulty making a selection. Let's just generate 100 rounds.")
    round_hold = 100 

#_____________________________________________________
# Result Mode
valid_results = ['D','S']
print("")
print("Last question:")
result_input = input('Do you want to (D)etail or (S)implified Results?  ')

# check for valid response
while result_input not in valid_results:
    print('Sorry, that was an invalid response. Please respond with a single letter.')
    result_input = input('Do you want to (D)etail or (S)implified Results?  ')


What would you like the player limit to be (dealer holds at 17)?  16
Thank you! Player will draw to 16.

How many rounds would you like simulated?  100
Thank you! 100 rounds will be simulated

Last question:
Do you want to (D)etail or (S)implified Results?  D


In [7]:
win_rates = {"PJDL":0, #Player blackjack, dealer lost
             "PJDB":0, #Player blackjack, dealer bust
             "PLDJ":0, #Player lost, dealer blackjack
             "PBDJ":0, #Player bust, dealer blackjack
             "PJDJ":0, #Push, each with blackjacks
             "PWDL":0, #Player wins, dealer lost
             "PWDB":0, #Player wins, dealer bust
             "PLDW":0, #Player loss, dealer wins
             "PBDW":0, #Player bust, dealer wins
             "PBDB":0, #Player bust, dealer bust
             "PpDp":0, #Standard Push
             "Errors":0} #Error Check

In [8]:
# randomly picks a shuffle trigger between the first and last 50 cards
cut_target = random.randint(49,(len(master_deck['suits']) - 51))

# Initial Setup
shuffles = 0
games = 0
dealer = new_deck()
player = new_deck()

# Start the deal timer
deal_start = time.time()

# loop for X rounds
for rounds in range(round_hold):
    #_____________________________________________________
    # Shuffle if cut target is reached
    
    if len(master_deck['suits']) <= cut_target:
        master_deck, discard_deck = rotary_shuffling(master_deck, discard_deck)
        shuffles += 1
        # generate new cut target
        cut_target = random.randint(49,(len(master_deck['suits']) - 51))
    #_____________________________________________________
    # Initial Draws
    
    for deal in range(2):
        for key in ["suits", "values", "ids"]:
            dealer[key].append(master_deck[key].pop(0))
        for key in ["suits", "values", "ids"]:
            player[key].append(master_deck[key].pop(0))
            
    #_____________________________________________________       
    # Double Ace checks
    if sum(player['values']) == 22:
            player_ace = 0 
            player['values'][player_ace] = 1 #Update dual ace deal before hand check
    if sum(dealer['values']) == 22:
        dealer_ace = 0 
        dealer['values'][dealer_ace] = 1 #Update dual ace deal before hand check
    #_____________________________________________________      
    # Check for instant loss condition, Skips to results if met    
    if sum(dealer['values']) != 21:
        
        #_____________________________________________________
        #Player Play

        while sum(player['values']) <= hand_hold:
            for key in ["suits", "values", "ids"]:
                player[key].append(master_deck[key].pop(0))
            
        while sum(player['values']) < hand_hold:
            for key in ["suits", "values", "ids"]:
                player[key].append(master_deck[key].pop(0))
            if sum(dealer['values']) > 21:
                if 11 in player['values']:
                    player_ace = player['values'].index(11)
                    player['values'][player_ace] = 1

        #_____________________________________________________
        #Dealer Play

        # draw to reach limit
                
        while sum(dealer['values']) < 17:
            for key in ["suits", "values", "ids"]:
                dealer[key].append(master_deck[key].pop(0))
            if sum(dealer['values']) > 21:
                if 11 in dealer['values']:
                    dealer_ace = dealer['values'].index(11)
                    dealer['values'][dealer_ace] = 1
                
    #_____________________________________________________
    
    # Round Results
    
    # Player blackjack
    if sum(player['values']) == 21 and len(player['values']) == 2:
        if sum(dealer['values']) == 21:
            if len(dealer['values']) == 2:
                win_rates["PJDJ"] += 1  # Push, each with blackjacks
            else:
                win_rates["PpDp"] += 1  # Standard Push
        elif sum(dealer['values']) > 21:
            win_rates["PJDB"] += 1  # Player blackjack, dealer bust
        elif sum(dealer['values']) < 21:
            win_rates["PJDL"] += 1  # Player blackjack, dealer lost
        else:
            win_rates["Errors"] += 1  # Error bin
            error_a += 1

    # Dealer blackjack        
    elif sum(dealer['values']) == 21 and len(dealer['values']) == 2:
        if sum(player['values']) != 21:
            win_rates["PLDJ"] += 1  # Player lost, dealer blackjack
        else:
            win_rates["Errors"] += 1  # Error bin
            error_b += 1

    # Player valid        
    elif sum(player['values']) <= 21: 
        if sum(dealer['values']) <= 21:
            if sum(player['values']) > sum(dealer['values']):
                win_rates["PWDL"] += 1  # Player wins, dealer lost
            elif sum(player['values']) < sum(dealer['values']):
                win_rates["PLDW"] += 1  # Player loss, dealer wins
            elif sum(dealer['values']) > 21:
                win_rates["PWDB"] += 1  # Player wins, dealer bust
            elif sum(player['values']) == sum(dealer['values']):
                win_rates["PpDp"] += 1  # Standard Push
            else:
                win_rates["Errors"] += 1  # Error bin
                error_c += 1
        else:
            win_rates["PWDB"] += 1  # Player wins, dealer bust
            
    # Player busts
    else:
        if sum(dealer['values']) <= 21:  # Dealer valid
            win_rates["PBDW"] += 1  # Player bust, dealer wins
        elif sum(dealer['values']) > 21:
            win_rates["PBDB"] += 1  # Player bust, dealer bust
        else:
            win_rates["Errors"] += 1  # Error bin
            error_d += 1
            
    # Ace reset
    if 1 in player['values']:
        player['values'][player_ace] = 11
    if 1 in dealer['values']:
        dealer['values'][dealer_ace] = 11

    # Discard Transfer
    for key, value in dealer.items():
        if key in discard_deck:
            discard_deck[key] += value

    for key, value in player.items():
        if key in discard_deck:
            discard_deck[key] += value

    # Bin resets
    dealer = new_deck()
    player = new_deck()

    games += 1
    
# End the deal timer
deal_end = time.time()

# RESULTS

In [9]:
print(f"Number of games = {games}")
print(f"Number of shuffles = {shuffles}")

Number of games = 100
Number of shuffles = 3


In [10]:
# Calculating sum of results
total_plays = 0
for key in win_rates:
    total_plays += win_rates[key]

print(f"Total games = {total_plays}") 
print("") # space
if result_input == "D":
    print(f"Player blackjack, dealer lost = {win_rates['PJDL']}") ##
    print(f"Player blackjack, dealer bust = {win_rates['PJDB']}") ##
    print(f"Player lost, dealer blackjack = {win_rates['PLDJ']}") ###
    print(f"Player bust, dealer blackjack = {win_rates['PBDJ']}") ###
    print(f"Push, each with blackjacks = {win_rates['PJDJ']}") #
    print(f"Player wins, dealer lost = {win_rates['PWDL']}") ##
    print(f"Player wins, dealer bust = {win_rates['PWDB']}") ##
    print(f"Player loss, dealer wins = {win_rates['PLDW']}") ###
    print(f"Player bust, dealer wins = {win_rates['PBDW']}") ###
    print(f"Player bust, dealer bust = {win_rates['PBDB']}") ###
    print(f"Standard Push = {win_rates['PpDp']}") #
    print(f"Error Count = {win_rates['Errors']}")
    print("") # space

win_rate = (win_rates['PJDL']+
            win_rates['PJDB']+
            win_rates['PWDL']+
            win_rates['PWDB'])
push_rate = (win_rates['PJDJ']+
             win_rates['PpDp'])
loss_rate = (win_rates['PLDJ']+
             win_rates['PBDJ']+
             win_rates['PLDW']+
             win_rates['PBDW']+
             win_rates['PBDB'])
print(f"Win Rate = {(win_rate/total_plays)*100}%")
print(f"Push Rate = {(push_rate/total_plays)*100}%")
print(f"Loss Rate = {(loss_rate/total_plays)*100}%")

Total games = 100

Player blackjack, dealer lost = 3
Player blackjack, dealer bust = 3
Player lost, dealer blackjack = 5
Player bust, dealer blackjack = 0
Push, each with blackjacks = 0
Player wins, dealer lost = 12
Player wins, dealer bust = 20
Player loss, dealer wins = 18
Player bust, dealer wins = 22
Player bust, dealer bust = 3
Standard Push = 14
Error Count = 0

Win Rate = 38.0%
Push Rate = 14.000000000000002%
Loss Rate = 48.0%


In [11]:
# End the full timer
boot_end = time.time()

# Calculate the elapsed times
deal_time = deal_end - deal_start

# Print the elapsed time
print("") # space
print(f"{games} games took: {deal_time} seconds")


100 games took: 0.003094196319580078 seconds


In [13]:
print(master_deck)

{'suits': ['♠8', '♦2', '♥3', '♦5', '♦6', '♦6', '♣6', '♦7', '♥A', '♣9', '♠3', '♠J', '♣Q', '♣6', '♣A', '♥3', '♠8', '♥10', '♣8', '♦7', '♥A', '♥4', '♦Q', '♦4', '♥8', '♣J', '♠9', '♥8', '♣8', '♣9', '♠6', '♥7', '♠J', '♥7', '♥6', '♠10', '♦5', '♣J', '♥9', '♠8', '♦6', '♣K', '♦4', '♠4', '♠7', '♥5', '♥5', '♠6', '♦6', '♦J', '♠6', '♥10', '♣Q', '♦10', '♠5', '♥9', '♥A', '♠8', '♣J', '♠10', '♦3', '♣2', '♠3', '♣5', '♥K', '♦A', '♦3', '♥7', '♥8', '♣8', '♠4', '♣10', '♠6', '♦Q', '♣A', '♣A', '♦7', '♣K', '♦K', '♥J', '♦6', '♥6', '♠J', '♠6', '♥8', '♥9', '♠6', '♦5', '♣7', '♣5', '♥2', '♦4', '♦3', '♥K', '♥10', '♦10', '♠Q', '♥9', '♣5', '♥10', '♠3', '♠9', '♠10', '♣5', '♦K', '♦9', '♠2', '♠K', '♥2', '♦2', '♠Q', '♥3', '♠7', '♣9', '♦2', '♦J', '♠2', '♥J', '♠10', '♣5', '♦K', '♦A', '♠K', '♥K', '♠K', '♥7', '♦3', '♣10', '♠K', '♥10', '♥K', '♦6', '♦7', '♣K', '♣Q', '♦6', '♦K', '♥9', '♠K', '♣10', '♣6', '♥2', '♣7', '♠3', '♥4', '♦J', '♣2', '♣2', '♣2', '♠Q', '♥4', '♥4', '♣9', '♠2', '♥J', '♣6', '♥4', '♦4', '♠7', '♥2', '♣6', '♣K', '♦Q